## Блокнот для прикидки влияния отдельных параметров


In [1]:
# from utils import X,y,cat_cols,fillna,normalize,skew,encode
from task import *
import gc
gc.collect()

1131

### Для тестирования используется тренировочный набор
Набор разделён на обучающий и тестовый  в соотношении 3:1 без перемешивания 

In [2]:
def data():
    split = 300_000
    maxlines = 500_000
    d2 = pd.read_csv('datasets/train.csv',skiprows= range(1,split) )
    d1 = pd.read_csv('datasets/train.csv',skiprows= range(split,maxlines) )
    return(d1,d2)  


### Класс для прогона тестов
На экран выводится метрика MAPE на тетсовом наборе 
Кроме того, базовый класс записывает метрику в лог, откуда её можно извлечь

In [3]:
class Loader(Task):

    def __init__( self, data, params = {}):
        super().__init__(data, params)
        self.ds = ScoreSet( data[0],data[1] ,params=self.params)

    def process(self,ds = None ):
        if ds==None:
            ds = self.ds
        ds.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        ds.prepare().fit().predict()
        self.log +=(f"{ds.log}\t MAPE:{ds.scores()}\n")          

    

### Загрузка модели

In [4]:
params = { 
            'rounding': { 'odometer':3000,'condition':1,'prefix_size':10},
            'model': {  'criterion':'poisson','n_estimators':200, 'max_depth':50, 'min_samples_leaf':2 },  
            'sellers':{'stages':5}
        }

v = Loader( data() ,params=params)
v.process()
print(v.log)       
        
            

	 MAPE:0.17088828637597805



In [5]:
import matplotlib.pyplot as  plt
global X

fis = v.ds.model.feature_importances_.argsort()
plt.barh(pd.Series([ x.upper() for x in X])[fis], v.ds.model.feature_importances_[fis]);
plt.xlabel("Влияние признаков на цену автомобиля");

KeyError: '[11, 10] not in index'

In [7]:
d = pd.read_csv('datasets/test.csv')

In [21]:
diff = pd.DataFrame({'predict':v.ds.pr,'price':v.ds.te.sellingprice} )

In [33]:
print(f"MAPE cars more expensive than 10_000: { mape(diff[diff.price>5000].predict,diff[diff.price>5000 ].price)} ")
print(f"MAPE for cars cheaper than 10_000: { mape(diff[diff.price<5000].predict,diff[diff.price<5000 ].price)} ")

MAPE cars more expensive than 10_000: 0.12634631539760965 
MAPE for cars cheaper than 10_000: 0.3632527411828443 


In [34]:
print( diff[diff.price>5000].price.count()/diff.count() )
print( diff[diff.price<5000].price.count()/diff.count() )

predict    0.810649
price      0.810649
dtype: float64
predict    0.18483
price      0.18483
dtype: float64
